In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


## import pwd
from config import password

In [2]:
#read csv into pandas
homeless_pop = pd.read_csv("homelessness.csv")

homeless_pop

,Year,State,CoC Number,CoC Name,Measures,Count
0,1/1/2007,AK,AK-500,Anchorage CoC,Chronically Homeless Individuals,224
1,1/1/2007,AK,AK-500,Anchorage CoC,Homeless Individuals,696
2,1/1/2007,AK,AK-500,Anchorage CoC,Homeless People in Families,278
3,1/1/2007,AK,AK-500,Anchorage CoC,Sheltered Chronically Homeless Individuals,187
4,1/1/2007,AK,AK-500,Anchorage CoC,Sheltered Homeless,842
...,...,...,...,...,...,...
86524,1/1/2016,WY,WY-500,Wyoming Statewide CoC,Unsheltered Homeless Unaccompanied Youth (Unde...,7
86525,1/1/2016,WY,WY-500,Wyoming Statewide CoC,Unsheltered Homeless Veterans,31
86526,1/1/2016,WY,WY-500,Wyoming Statewide CoC,Unsheltered Parenting Youth (Under 25),3
86527,1/1/2016,WY,WY-500,Wyoming Statewide CoC,Unsheltered Parenting Youth Age 18-24,3


In [3]:
#identify the data that is FL only
fl_homeless_pop = homeless_pop.loc[homeless_pop["State"] == "FL"]
fl_homeless_pop

#take commas out of the count column
fl_homeless_pop["Count"] = fl_homeless_pop["Count"].str.replace(",","")
fl_homeless_pop["Count"] = pd.to_numeric(fl_homeless_pop["Count"])


<ipython-input-3-858244bd2eaf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_homeless_pop["Count"] = fl_homeless_pop["Count"].str.replace(",","")
<ipython-input-3-858244bd2eaf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_homeless_pop["Count"] = pd.to_numeric(fl_homeless_pop["Count"])


In [4]:
fl_homeless_pop["Measures"] = fl_homeless_pop["Measures"].str.replace(" ","_")
fl_homeless_pop

<ipython-input-4-07d7b7b0f426>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_homeless_pop["Measures"] = fl_homeless_pop["Measures"].str.replace(" ","_")


,Year,State,CoC Number,CoC Name,Measures,Count
828,1/1/2007,FL,FL-500,"Sarasota/Bradenton/Manatee, Sarasota Counties CoC",Chronically_Homeless_Individuals,135
829,1/1/2007,FL,FL-500,"Sarasota/Bradenton/Manatee, Sarasota Counties CoC",Homeless_Individuals,777
830,1/1/2007,FL,FL-500,"Sarasota/Bradenton/Manatee, Sarasota Counties CoC",Homeless_People_in_Families,235
831,1/1/2007,FL,FL-500,"Sarasota/Bradenton/Manatee, Sarasota Counties CoC",Sheltered_Chronically_Homeless_Individuals,0
832,1/1/2007,FL,FL-500,"Sarasota/Bradenton/Manatee, Sarasota Counties CoC",Sheltered_Homeless,494
...,...,...,...,...,...,...
73714,1/1/2016,FL,FL-606,Naples/Collier County CoC,Unsheltered_Homeless_Unaccompanied_Youth_(Unde...,11
73715,1/1/2016,FL,FL-606,Naples/Collier County CoC,Unsheltered_Homeless_Veterans,1
73716,1/1/2016,FL,FL-606,Naples/Collier County CoC,Unsheltered_Parenting_Youth_(Under_25),0
73717,1/1/2016,FL,FL-606,Naples/Collier County CoC,Unsheltered_Parenting_Youth_Age_18-24,0


In [5]:
#pivot the table so the measures are the headers instead of rows
fl_homeless_new = fl_homeless_pop.pivot_table(index=["Year", "State", "CoC Number", "CoC Name"],values = ["Count"], columns=["Measures"], aggfunc=np.max)
fl_homeless_new.head()

Count  \
Measures                                                                     Children_of_Parenting_Youth   
Year     State CoC Number CoC Name                                                                         
1/1/2007 FL    FL-500     Sarasota/Bradenton/Manatee, Sarasota Counties CoC                          NaN   
               FL-501     Tampa/Hillsborough County CoC                                              NaN   
               FL-502     St. Petersburg/Clearwater/Largo/Pinellas County...                         NaN   
               FL-503     Lakeland/Winter Haven/Polk County CoC                                      NaN   
               FL-504     Daytona Beach/Daytona/Volusia, Flagler Counties...                         NaN   

                                                                                                   \
Measures                                                                     Chronically_Homeless   
Year     State CoC Number CoC Name                                                                  
1/1/2007 FL    FL-500     Sarasota/Bradenton/Manatee, Sarasota Counties CoC                   NaN   
               FL-501     Tampa/Hillsborough County CoC                                       NaN   
               FL-502     St. Petersburg/Clearwater/Largo/Pinellas County...                  NaN   
               FL-503     Lakeland/Winter Haven/Polk County CoC                               NaN   
               FL-504     Daytona Beach/Daytona/Volusia, Flagler Counties...                  NaN   

                                                                                                               \
Measures                                                                     Chronically_Homeless_Individuals   
Year     State CoC Number CoC Name                                                                              
1/1/2007 FL    FL-500     Sarasota/Bradenton/Manatee, Sarasota Counties CoC                             135.0   
               FL-501     Tampa/Hillsborough County CoC                                                 528.0   
               FL-502     St. Petersburg/Clearwater/Largo/Pinellas County...                            611.0   
               FL-503     Lakeland/Winter Haven/Polk County CoC                                         197.0   
               FL-504     Daytona Beach/Daytona/Volusia, Flagler Counties...                            184.0   

                                                                                                                      \
Measures                                                                     Chronically_Homeless_People_in_Families   
Year     State CoC Number CoC Name                                                                                     
1/1/2007 FL    FL-500     Sarasota/Bradenton/Manatee, Sarasota Counties CoC                                      NaN   
               FL-501     Tampa/Hillsborough County CoC                                                          NaN   
               FL-502     St. Petersburg/Clearwater/Largo/Pinellas County...                                     NaN   
               FL-503     Lakeland/Winter Haven/Polk County CoC                                                  NaN   
               FL-504     Daytona Beach/Daytona/Volusia, Flagler Counties...                                     NaN   

                                                                                                   \
Measures                                                                     Homeless_Individuals   
Year     State CoC Number CoC Name                                                                  
1/1/2007 FL    FL-500     Sarasota/Bradenton/Manatee, Sarasota Counties CoC                 777.0   
               FL-501     Tampa/Hillsborough County CoC                                    4397.0   
               FL-502     St. Petersburg/Clearwater/La

In [6]:
#reset df.columns 
fl_homeless_new.columns = fl_homeless_new.columns.get_level_values(1)

In [7]:
fl_homeless_new.columns[0]

'Children_of_Parenting_Youth'

In [8]:
fl_homeless_new.columns

Index(['Children_of_Parenting_Youth', 'Chronically_Homeless',
       'Chronically_Homeless_Individuals',
       'Chronically_Homeless_People_in_Families', 'Homeless_Individuals',
       'Homeless_People_in_Families',
       'Homeless_Unaccompanied_Children_(Under_18)',
       'Homeless_Unaccompanied_Young_Adults_(Age_18-24)',
       'Homeless_Unaccompanied_Youth_(Under_25)', 'Homeless_Veterans',
       'Parenting_Youth_(Under_25)', 'Parenting_Youth_Age_18-24',
       'Parenting_Youth_Under_18', 'Sheltered_Children_of_Parenting_Youth',
       'Sheltered_Chronically_Homeless',
       'Sheltered_Chronically_Homeless_Individuals',
       'Sheltered_Chronically_Homeless_People_in_Families',
       'Sheltered_Homeless', 'Sheltered_Homeless_Individuals',
       'Sheltered_Homeless_People_in_Families',
       'Sheltered_Homeless_Unaccompanied_Children_(Under_18)',
       'Sheltered_Homeless_Unaccompanied_Young_Adults_(Age_18-24)',
       'Sheltered_Homeless_Unaccompanied_Youth_(Under_25)',
   

In [9]:
#connect to the postgress engine
postgres_conn_string = f"postgres:{password}" 
engine = create_engine(f"postgresql://{postgres_conn_string}@localhost:5432/housing", paramstyle="format")

In [10]:
#check for the tables that should already be in postgress
engine.table_names()
# df = pd.read_sql('select * from homelessness', engine)
# df.columns

['homelessness', 'incomes', 'housing', 'rental_pricing']

In [11]:
fl_homeless_new = fl_homeless_new.reset_index()
fl_homeless_new.head()

Measures,Year,State,CoC Number,CoC Name,Children_of_Parenting_Youth,Chronically_Homeless,Chronically_Homeless_Individuals,Chronically_Homeless_People_in_Families,Homeless_Individuals,Homeless_People_in_Families,...,Unsheltered_Homeless,Unsheltered_Homeless_Individuals,Unsheltered_Homeless_People_in_Families,Unsheltered_Homeless_Unaccompanied_Children_(Under_18),Unsheltered_Homeless_Unaccompanied_Young_Adults_(Age_18-24),Unsheltered_Homeless_Unaccompanied_Youth_(Under_25),Unsheltered_Homeless_Veterans,Unsheltered_Parenting_Youth_(Under_25),Unsheltered_Parenting_Youth_Age_18-24,Unsheltered_Parenting_Youth_Under_18
0,1/1/2007,FL,FL-500,"Sarasota/Bradenton/Manatee, Sarasota Counties CoC",NaN,NaN,135.0,NaN,777.0,235.0,...,518.0,435.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/1/2007,FL,FL-501,Tampa/Hillsborough County CoC,NaN,NaN,528.0,NaN,4397.0,2086.0,...,5433.0,3813.0,1620.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1/1/2007,FL,FL-502,St. Petersburg/Clearwater/Largo/Pinellas Count...,NaN,NaN,611.0,NaN,2129.0,397.0,...,1221.0,1218.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/1/2007,FL,FL-503,Lakeland/Winter Haven/Polk County CoC,NaN,NaN,197.0,NaN,593.0,209.0,...,315.0,257.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/1/2007,FL,FL-504,"Daytona Beach/Daytona/Volusia, Flagler Countie...",NaN,NaN,184.0,NaN,854.0,624.0,...,909.0,651.0,258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
fl_homeless_new.columns
columns = [col for col in fl_homeless_new.columns if col not in ['level_0', 'index']]
fl_homeless_new = fl_homeless_new[columns]

In [13]:
fl_homeless_new = fl_homeless_new.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'CoC Number': 'CoC_number',
    'CoC Name': 'CoC_name'
})

fl_homeless_new['year'] = pd.DatetimeIndex(fl_homeless_new['year']).year

In [14]:
fl_homeless_new.to_sql(name='homelessness', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_homelessness"
DETAIL:  Key (year)=(2007) already exists.

[SQL: INSERT INTO homelessness (year, state, "CoC_number", "CoC_name", "Children_of_Parenting_Youth", "Chronically_Homeless", "Chronically_Homeless_Individuals", "Chronically_Homeless_People_in_Families", "Homeless_Individuals", "Homeless_People_in_Families", "Homeless_Unaccompanied_Children_(Under_18)", "Homeless_Unaccompanied_Young_Adults_(Age_18-24)", "Homeless_Unaccompanied_Youth_(Under_25)", "Homeless_Veterans", "Parenting_Youth_(Under_25)", "Parenting_Youth_Age_18-24", "Parenting_Youth_Under_18", "Sheltered_Children_of_Parenting_Youth", "Sheltered_Chronically_Homeless", "Sheltered_Chronically_Homeless_Individuals", "Sheltered_Chronically_Homeless_People_in_Families", "Sheltered_Homeless", "Sheltered_Homeless_Individuals", "Sheltered_Homeless_People_in_Families", "Sheltered_Homeless_Unaccompanied_Children_(Under_18)", "Shelt

In [ ]:
pd.read_sql_query('select * from homelessness', con=engine).head()